In [1]:
import requests
import psycopg2
import pandas as pd

Create Database in Postgres

In [2]:
def create_database():
    # Connect to default database
    conn = psycopg2.connect("host=localhost port=5432 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create nba_transactions database
    cur.execute("DROP DATABASE IF EXISTS nba_transactions;")
    cur.execute("CREATE DATABASE nba_transactions;")

    # Close connection to default DB
    conn.close()

    # Connect to nba_transactions DB
    conn = psycopg2.connect("host=localhost port=5432 dbname=nba_transactions user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    return conn, cur

Extract data from ESPN

In [3]:

url = "https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/transactions"

querystring = {"region":"us","lang":"en","contentorigin":"espn","limit":"1000","page":"1"}

payload = ""
headers = {
    "authority": "site.web.api.espn.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,he-IL;q=0.8,he;q=0.7",
    "origin": "https://www.espn.com",
    "referer": "https://www.espn.com/",
    "sec-ch-ua": "^\^Not.A/Brand^^;v=^\^8^^, ^\^Chromium^^;v=^\^114^^, ^\^Google",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

r = requests.request("GET", url, data=payload, headers=headers, params=querystring)


In [4]:
# change output format into json.
data = r.json()

In [5]:
data.keys()

dict_keys(['timestamp', 'status', 'season', 'requestedYear', 'count', 'pageIndex', 'pageSize', 'pageCount', 'transactions'])

In [6]:
trans_data = data['transactions']

In [7]:
#transaction date
trans_data[0]['date']

'2023-06-30T07:00Z'

In [8]:
#description
trans_data[0]['description']

'Waived F Trendon Watford.'

In [9]:
#team id
trans_data[0]['team']

{'id': '22',
 'location': 'Portland',
 'name': 'Trail Blazers',
 'abbreviation': 'POR',
 'displayName': 'Portland Trail Blazers',
 'color': 'e03a3e',
 'alternateColor': '000000',
 'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/nba/500/por.png',
   'width': 500,
   'height': 500,
   'alt': '',
   'rel': ['full', 'default'],
   'lastUpdated': '2018-06-05T12:08Z'},
  {'href': 'https://a.espncdn.com/i/teamlogos/nba/500-dark/por.png',
   'width': 500,
   'height': 500,
   'alt': '',
   'rel': ['full', 'dark'],
   'lastUpdated': '2018-06-05T12:08Z'},
  {'href': 'https://a.espncdn.com/i/teamlogos/nba/500/scoreboard/por.png',
   'width': 500,
   'height': 500,
   'alt': '',
   'rel': ['full', 'scoreboard'],
   'lastUpdated': '2018-06-05T12:08Z'},
  {'href': 'https://a.espncdn.com/i/teamlogos/nba/500-dark/scoreboard/por.png',
   'width': 500,
   'height': 500,
   'alt': '',
   'rel': ['full', 'scoreboard', 'dark'],
   'lastUpdated': '2018-06-05T12:08Z'}],
 'links': [{'language': 'en-US',

In [10]:
#team name
trans_data[0]['team']['name']

'Trail Blazers'

In [11]:
#team website
trans_data[0]['team']['links'][0]['href']

'https://www.espn.com/nba/team/_/name/por/portland-trail-blazers'

In [12]:
# transaction table (transaction_date, team_name, description)
transactions_list = []

for row in trans_data:
    transaction_date = row['date']
    team_name = row['team']['displayName']
    description = row['description']

    transactions_element = {'transaction_date': transaction_date, 'team_name': team_name, 'description': description}
    
    transactions_list.append(transactions_element)
    

In [13]:
df_transactions = pd.DataFrame.from_dict(transactions_list)

In [14]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_date  114 non-null    object
 1   team_name         114 non-null    object
 2   description       114 non-null    object
dtypes: object(3)
memory usage: 2.8+ KB


In [15]:
# change 'transaction_date' data type to datetime.
df_transactions['transaction_date'] = pd.to_datetime(df_transactions['transaction_date'])

In [16]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   transaction_date  114 non-null    datetime64[ns, UTC]
 1   team_name         114 non-null    object             
 2   description       114 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 2.8+ KB


In [17]:
# team table (team_id, team_name, displayname, team_web )
team_list = []

for row in trans_data:
    id = row['team']['id']
    name = row['team']['name']
    displayname = row['team']['displayName']
    abbreviation = row['team']['abbreviation']
    website = row['team']['links'][0]['href']

    team_element = {'id': id, 'name': name, 'displayname': displayname, 'abbreviation': abbreviation, 'website': website}

    team_list.append(team_element)

In [18]:
df_teams = pd.DataFrame.from_dict(team_list)

In [19]:
df_teams.head()

,id,name,displayname,abbreviation,website
0,22,Trail Blazers,Portland Trail Blazers,POR,https://www.espn.com/nba/team/_/name/por/portl...
1,16,Timberwolves,Minnesota Timberwolves,MIN,https://www.espn.com/nba/team/_/name/min/minne...
2,13,Lakers,Los Angeles Lakers,LAL,https://www.espn.com/nba/team/_/name/lal/los-a...
3,16,Timberwolves,Minnesota Timberwolves,MIN,https://www.espn.com/nba/team/_/name/min/minne...
4,29,Grizzlies,Memphis Grizzlies,MEM,https://www.espn.com/nba/team/_/name/mem/memph...


In [20]:
df_teams.drop_duplicates(subset=['id'],inplace=True)

In [21]:
df_teams.reset_index(drop=True,inplace=True)

In [22]:
df_teams.head()

,id,name,displayname,abbreviation,website
0,22,Trail Blazers,Portland Trail Blazers,POR,https://www.espn.com/nba/team/_/name/por/portl...
1,16,Timberwolves,Minnesota Timberwolves,MIN,https://www.espn.com/nba/team/_/name/min/minne...
2,13,Lakers,Los Angeles Lakers,LAL,https://www.espn.com/nba/team/_/name/lal/los-a...
3,29,Grizzlies,Memphis Grizzlies,MEM,https://www.espn.com/nba/team/_/name/mem/memph...
4,27,Wizards,Washington Wizards,WSH,https://www.espn.com/nba/team/_/name/wsh/washi...


Create tables in nba_transactions Database

In [23]:
conn, cur = create_database()

# Create table transactions
transactions_create_table = ("""CREATE TABLE IF NOT EXISTS transactions
                    (transaction_date timestamp,
                    team_name VARCHAR,
                    description VARCHAR
                    )""")
cur.execute(transactions_create_table)

# Create table teams
teams_create_table = ("""CREATE TABLE IF NOT EXISTS teams
                    (id INT PRIMARY KEY,
                    name VARCHAR,
                    displayname VARCHAR,
                    abbreviation VARCHAR,
                    website VARCHAR
                    )""")
cur.execute(teams_create_table)


Insert values from Pandas to Postgres

In [24]:
transactions_table_insert = ("""INSERT INTO transactions(
                    transaction_date,
                    team_name,
                    description)
                    VALUES (%s, %s, %s)
                    """)

In [25]:
try:
    for i, row in df_transactions.iterrows():
        cur.execute(transactions_table_insert, list(row))
        
except psycopg2.Error as e:
    print(e)

In [26]:
teams_table_insert = ("""INSERT INTO teams(
                    id,
                    name,
                    displayname,
                    abbreviation,
                    website)
                    VALUES (%s, %s, %s, %s, %s)
                    """)

In [27]:
try:
    for i, row in df_teams.iterrows():
        cur.execute(teams_table_insert, list(row))
        
except psycopg2.Error as e:
    print(e)